In [6]:
import glob, json

data_files = "data/*.txt"
text = ""
files = glob.glob(data_files)
for name in files:
    try:
        with open(name) as f:
            text += f.read()
    except IOError as exc:
        if exc.errno != errno.EISDIR:
            raise
            
text

## Tokenization with Indicnlp Library

In [5]:
from indicnlp.tokenize import sentence_tokenize, indic_tokenize

def awadhi_tokenizer(text):
    sentences = sentence_tokenize.sentence_split(text, lang='awa')
    tokenized_sentences = []
    for sentence in sentences:
        words = list(indic_tokenize.trivial_tokenize(sentence))
        tokenized_sentences.append(words)
    return {
        "sentences": sentences,
        "tokenized_sentences": tokenized_sentences,
    }

result = awadhi_tokenizer(text)

print("Tokenized Sentences:")
for sentence in result['sentences']:
    print(sentence)

print("\nTokenized Words:")
for words in result['tokenized_sentences']:
    print(words)

Tokenized Sentences:
महतारी बनैक सुख कैसे मिली ?
हमका धोखा काहे दीन्हेव ?
हनुमान दादा के पास कौनौ जवाब न रहै ।
उइ बसि चन्दावती का टुकुर-टुकुर द्याखति रहे ।
माफी माँगै के अलावा उनके पास कोनौ रस्ता न रहै ।
राति मा दवाई क्यार कउनौ कमाल न भवा रहै ।
हनुमान दादा राति भर जागति रहे उनकी आँखी अंगार हुइ गयी रहैं ।
" बहुत दिन तक दुनहू के बीच तना तनी रही बादि मा धीरे धीरे चन्दावती अपने मन का समझाय लीन्हेसि रहै - कि बेऔलाद सही सुकुलाइन तो हम बनि ही गयेन ।
गाँव के मनई सुकुलाइनि समझिके इज्जति करति हैं औ सिवपरसाद हमरे लरिकै तना तो है ।
आजु उनकी तेरही के बादि चन्दावती घर मा बिल्कुल अकेलि हुइ गै रहै ।
छोटकउनू तो अल्टीमेटम दै दीन्हेनि रहै कि अब घर ते निकरि जाव काहेते , जी तुमका लाये रहै उइ अब खतम हुइगे ।
कोई औलादि होतै तो छोटकउनू या हिम्मति के पउतीं ?
तेरही के दिनै संझा का ईतना छोटकउनू चन्दावती ते कहेनि तौ चन्दावती क मन कटी कनकइया तना काँपै लाग ,वा बहुत दुखी हुइगै रहै ।
बेऔलाद होयक दुखु ,पति के न रहै क्यार दुखु ,तीकै अब घर ते बेघर होयक दुखु चन्दावती क करेजु छलनी किहे देति रहै ।
क वहिके आगे पंचाइति करैके 

In [10]:
with open("emoji_sentiment_map.json", "r") as file:
    emoji_sentiment_map = json.load(file)

In [63]:
import re
from collections import Counter
from itertools import combinations
from itertools import tee
import emoji


def replace_emojis_with_sentiments(text, sentiment_map):
    emojis_in_text = emoji.emoji_list(text)
    
    # Replace each emoji with its sentiment
    for emj_dict in emojis_in_text:
        emj = emj_dict['emoji']
        sentiment = sentiment_map.get(emj, "neutral")  # Default to 'neutral' if emoji not found
        text = text.replace(emj, sentiment)
    
    return text

def english_tokenizer_no_numbers(text):
    text = text.lower()
    text = re.sub(r'\d+', '', text)  # Remove all numbers
    text = re.sub(r'[^\w\s]', '', text)  # Remove punctuation except underscores
    tokens = text.split()
    return tokens


def english_tokenizer(text):
    # Define the regular expression pattern
    pattern = r"""
        (?<!\S)[a-zA-Z]+(?:'[a-z]+)?    # Words with optional contractions
        | [.,!?;:"'()]                  # Punctuation marks
        | [@#$%^&*<>/\|~`+=-_]          # Special characters
    """

    text = replace_emojis_with_sentiments(text, emoji_sentiment_map)
    # Compile the pattern with verbose flag
    regex = re.compile(pattern, re.VERBOSE)
    # Find all matches
    tokens = regex.findall(text)
    unique_tokens = list(set(tokens))
    return unique_tokens

def generate_bigrams(tokens):
    a, b = tee(tokens)
    next(b, None)
    return list(zip(a, b))

def find_top_n_bigrams(tokens, n):
    bigrams = generate_bigrams(tokens)
    bigram_counts = Counter(bigrams)
    return bigram_counts.most_common(n)

def replace_bigrams_with_tokens(text, top_n_bigrams):
    # Convert each bigram into a single token (e.g., ("New", "York") -> "New York")
    bigram_tokens = [' '.join(bigram) for bigram, _ in top_n_bigrams]
    
    # Replace each occurrence of the bigram in the text with the combined token
    for bigram_token in bigram_tokens:
        bigram_pattern = r'\b' + re.escape(bigram_token) + r'\b'
        text = re.sub(bigram_pattern, bigram_token.replace(' ', '_'), text)
    
    return text
def english_tokenizer_with_bigrams(text, top_n_bigrams):
    text = replace_bigrams_with_tokens(text, top_n_bigrams)
    return english_tokenizer_no_numbers(text)

def tokenizer(text):
    tokens = english_tokenizer(text)
    top_n_bigrams = find_top_n_bigrams(tokens, n=3)
    updated_text = replace_bigrams_with_tokens(text, top_n_bigrams)
    updated_vocab = english_tokenizer_with_bigrams(updated_text, top_n_bigrams)
    return updated_vocab
    
# tokens = english_tokenizer(text)
# print("Total Unique Tokens: ", len(tokens))

# top_n_bigrams = find_top_n_bigrams(tokens, n=3)
# print("Top 3 Frequent Bigrams:", top_n_bigrams)

# # Step 4: Add top N bigrams to the vocabulary
# updated_vocab = add_top_n_bigrams_to_vocab(tokens, top_n_bigrams)
# print("Updated Vocabulary:", updated_vocab)


In [62]:
tokenizer("sf 🙄")

NameError: name 'english_tokenizer_no_numbers' is not defined